In [4]:
# from fastai import learner
from fastai.vision.all import *
import numpy as np
import os
import pandas as pd
import librosa as lb
import librosa.display
import soundfile as sf
import matplotlib.patches as patch
import matplotlib.pyplot as plt
import IPython.display as ipd
from pathlib import Path
from tqdm.notebook import tqdm
from scipy.io import wavfile
from torch import cuda
import shutil

CC_ROOT=Path("D:\\KidElectric\\rfcx_kaggle") #Cookiecutter datascience-style project
CC_DATA_TEST=CC_ROOT.joinpath('data').joinpath('interim').joinpath('test')
CC_MODEL_ROOT=CC_ROOT.joinpath('models').joinpath('fit')
df_train = pd.DataFrame({
    "recording_id": [path.stem for path in TRAIN_AUDIO_ROOT.glob("*.flac")],
})

df_test = pd.DataFrame({
    "recording_id": [path.stem for path in TEST_AUDIO_ROOT.glob("*.flac")],
})

df_tp=pd.read_csv(CC_ROOT.joinpath('references').joinpath('train_tp.csv')).set_index('recording_id')

df_fp=pd.read_csv(CC_ROOT.joinpath('references').joinpath('train_fp.csv')).set_index('recording_id')


In [6]:
#Load a model:
def which_singer(x): return x.parts[-1].split('_')[1] #Need to define this function first
model_name='rn101_50ep_512_clip_full_aug2_noflip_FB_120320_mixup0p5.pkl'
learn=load_learner(CC_MODEL_ROOT.joinpath(model_name))

In [10]:
recs=df_test['recording_id']
pngs=[png for png in CC_DATA_TEST.joinpath('clip_decomp').glob(recs[0] + '*.png')]
print(pngs)
print(recs[0])
print(len(pngs))

[Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_0.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_1.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_2.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_3.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_4.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_5.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_6.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_7.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_8.png'), Path('D:/KidElectric/rfcx_kaggle/data/interim/test/clip_decomp/000316da7_9.png')]
000316da7
10


In [14]:
CC_MODEL_ROOT.parent.joinpath('predict').joinpath('%s.csv' % model_name[0:-4])

Path('D:/KidElectric/rfcx_kaggle/models/predict/rn101_50ep_512_clip_full_aug2_noflip_FB_120320_mixup0p5.csv')

In [16]:
learn.dls.vocab

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '3', '4', '5', '6', '7', '8', '9', '99']

In [ ]:
#Score 10 spectrogram clips per testrec using one model:
pngs=[png for png in CC_DATA_TEST.joinpath('clip_decomp').glob('*.png')]
df_prob=pd.DataFrame(index=df_test['recording_id'],columns=['s%d' % i for i in range(0,24,1)])
prob_order= learn.dls.vocab
recs=df_test['recording_id']

for rec in recs:
    pngs=[png for png in CC_DATA_TEST.joinpath('clip_decomp').glob(rec + '*.png')]
    temp=np.zeros((len(pngs),len(prob_order)))
    for i,png in enumerate(pngs):
        fn=png.parts[-1]
        _,_,probs=learn.predict(png)
        temp[i,:]=probs
    mp=np.sum(temp,axis=0) #Or should it be mean?
    for i,spec in enumerate(prob_order):
        if int(spec) < 24:
            df_prob['s%s' % spec][rec]=float(mp[i])
df_prob.to_csv(CC_MODEL_ROOT.parent.joinpath('predict').joinpath('%s.csv' % model_name[0:-4]))
print('Finished')

In [23]:
# Predict many images at once? 
imgs=[png for png in CC_DATA_TEST.joinpath('clip_decomp').glob('*.png')]
dl = learn.dls.test_dl(imgs)
inp, preds,_,dec_preds = learner.get_preds(dl=dl, with_input=True, with_decoded=True)
full_dec = learner.dls.decode_batch((*tuplify(inp),*tuplify(dec_preds)))

AttributeError: type object 'ImageDataLoaders' has no attribute 'from_path_'